In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import dateutil as du
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
import multiprocessing as mp
%pylab inline
exec(open("/Users/raghuramkowdeed/Documents/afp/models/quintile_model.py").read())
exec(open("/Users/raghuramkowdeed/Documents/afp/get_signal_names.py").read())



Populating the interactive namespace from numpy and matplotlib


In [2]:
#data = pd.read_csv('/Users/raghuramkowdeed/Desktop/rps_data_clean.csv')
#data = data.set_index(['date'])
#data.index = [ dt.datetime.strptime(str(x),'%Y%m%d') for x in data.index ]

In [3]:
#data.to_pickle('/Users/raghuramkowdeed/Desktop/clean_rps_data.pkl')

In [4]:
#df = pd.read_pickle('/Users/raghuramkowdeed/Desktop/clean_rps_data.pkl')

In [5]:
def compute_signal_ic(arg):
    sig_name = arg[0]
    gamma = arg[1]
    look_back = arg[2]

    data = pd.read_pickle('/Users/raghuramkowdeed/Desktop/clean_rps_data.pkl')
    rebalance_dates = np.unique(data.index)
    rebalance_dates = np.sort( rebalance_dates )
    
    l_ic_vec = []
    nl_ic_vec = []
    comb_ic_vec = []
    nl_l_corr_vec = []
    model_vec = []
    train_x_vec = []
    
    for ind in range( 200, rebalance_dates.shape[0] ) :
        r_d = rebalance_dates[ind]
        print(r_d)
        train_end_date = rebalance_dates[ind-1] 
        #train_end_year = train_end_date.year

        train_start_date = rebalance_dates[ind-look_back]
        #train_start_year = train_start_date.year

        #curr_year = r_d.year

        train_data = data
        train_data = train_data[train_data.index>=train_start_date]
        train_data = train_data[train_data.index<=train_end_date]

        test_data = data
        test_data = test_data[test_data.index==r_d]

        train_x = train_data[ [sig_name]]
        train_y = train_data['ret']
        test_x = test_data[[sig_name]]
        test_y = test_data['ret']

        
        #model = QuintileSort(num_buckets,'median')
        #model = KNeighborsRegressor(n_neighbors=int(train_data.shape[0]*1.0/num_buckets)) 
        
        
        #print(train_x.shape)
        #print(train_x.index[0])
        #print(train_x.index[-1])
        
        #print(test_x.shape)
        #print(test_x.index[0])
        #print(test_x.index[-1])
        #print('-------------')
        
        model = KernelRidge(kernel='rbf', gamma=0.1)
        
        
        model.fit(train_x,train_y)

        #model.ret_profile.plot(kind='bar')

        
        nl_pred_ret = model.predict(test_x)
        nl_pred_ret = pd.Series(nl_pred_ret, index=test_x.index)
        nl_pred_ret = nl_pred_ret/nl_pred_ret.std()

        nl_corr = nl_pred_ret.corr(test_y)

        lm_model = LinearRegression(fit_intercept=False)
        lm_model.fit(train_x, train_y)
        #beta = (train_x.iloc[:,0]).corr(train_y.iloc[:,0]) * (train_y.iloc[:,0]).std() / (train_x[:,0]).std()
        l_pred_ret = lm_model.predict(test_x)
        l_pred_ret = pd.Series(l_pred_ret, index=test_x.index)
        l_pred_ret = l_pred_ret/l_pred_ret.std()

        l_corr = l_pred_ret.corr(test_y)

        comb_pred_ret = ( nl_pred_ret + l_pred_ret )/2.0
        comb_corr = comb_pred_ret.corr(test_y)


        nl_l_corr = nl_pred_ret.corr(l_pred_ret)


        l_ic_vec.append(l_corr)
        nl_ic_vec.append(nl_corr)

        comb_ic_vec.append(comb_corr)

        nl_l_corr_vec.append(nl_l_corr)

        model_vec.append(model)
        train_x_vec.append(train_x)

        #print( r_d,l_corr, nl_corr )

    df = pd.DataFrame()
    df['l_corr'] = l_ic_vec
    df['nl_corr'] = nl_ic_vec
    df['comb_ic'] = comb_ic_vec
    df['nl_l_corr'] = nl_l_corr_vec

    corr_s = df.mean(axis=0)
    corr_s.name = sig_name
    #corr_df = corr_df.append(corr_s)
    print(sig_name)
    print([corr_s.iloc[0], corr_s.iloc[1]])
    return([corr_s.iloc[0], corr_s.iloc[1]])

In [6]:
#arg = ['cashdebt', 0.1, 12]

#res = compute_signal_ic(arg)

#res

In [8]:
sig_names = get_signal_names()
sig_names = ['roic']
gamma = 0.1
look_back = 1
arg_vec = [ [val, gamma, look_back] for val in sig_names ]
#pool = mp.Pool(processes=2)
#results = pool.map(compute_signal_ic, arg_vec)
results = [ compute_signal_ic(val) for val in arg_vec]
print(results)


1996-09-30T00:00:00.000000000
1996-10-31T00:00:00.000000000
1996-11-29T00:00:00.000000000
1996-12-31T00:00:00.000000000
1997-01-31T00:00:00.000000000
1997-02-28T00:00:00.000000000
1997-03-31T00:00:00.000000000
1997-04-30T00:00:00.000000000
1997-05-30T00:00:00.000000000
1997-06-30T00:00:00.000000000
1997-07-31T00:00:00.000000000
1997-08-29T00:00:00.000000000
1997-09-30T00:00:00.000000000
1997-10-31T00:00:00.000000000
1997-11-28T00:00:00.000000000
1997-12-31T00:00:00.000000000
1998-01-30T00:00:00.000000000
1998-02-27T00:00:00.000000000
1998-03-31T00:00:00.000000000
1998-04-30T00:00:00.000000000
1998-05-29T00:00:00.000000000
1998-06-30T00:00:00.000000000
1998-07-31T00:00:00.000000000
1998-08-31T00:00:00.000000000
1998-09-30T00:00:00.000000000
1998-10-30T00:00:00.000000000
1998-11-30T00:00:00.000000000
1998-12-31T00:00:00.000000000
1999-01-29T00:00:00.000000000
1999-02-26T00:00:00.000000000
1999-03-31T00:00:00.000000000
1999-04-30T00:00:00.000000000
1999-05-28T00:00:00.000000000
1999-06-30

In [22]:
mp.cpu_count()


4

In [6]:
start_year = 2000
start_date = dt.datetime.strptime(str(start_year)+'0101','%Y%m%d')
end_year = 2015
end_date = dt.datetime.strptime(str(end_year)+'0101','%Y%m%d')

In [7]:
look_back = 1*1
sig_names = get_signal_names()
sig_names = sig_names[0::5]
#sig_names = ['cashdebt']
num_buckets = 5
gamma = 1.0
rebalance_dates = np.unique(data.index)

In [8]:
corr_df = pd.DataFrame()  
for sig_name in sig_names:
    l_ic_vec = []
    nl_ic_vec = []
    comb_ic_vec = []
    nl_l_corr_vec = []
    model_vec = []
    train_x_vec = []
    
    for ind in range( 200, rebalance_dates.shape[0] ) :
        r_d = rebalance_dates[ind]
        #print(r_d)
        train_end_date = rebalance_dates[ind-1] 
        #train_end_year = train_end_date.year

        train_start_date = rebalance_dates[ind-look_back]
        #train_start_year = train_start_date.year

        #curr_year = r_d.year

        train_data = data
        train_data = train_data[train_data.index>=train_start_date]
        train_data = train_data[train_data.index<=train_end_date]

        test_data = data
        test_data = test_data[test_data.index==r_d]


        #model = QuintileSort(num_buckets,'median')
        #model = KNeighborsRegressor(n_neighbors=int(train_data.shape[0]*1.0/num_buckets)) 
        model = KernelRidge(kernel='rbf', gamma=0.1)
        train_x = train_data[ [sig_name]]
        train_y = train_data['ret']

        
        model.fit(train_x,train_y)

        #model.ret_profile.plot(kind='bar')

        test_x = test_data[[sig_name]]
        test_y = test_data['ret']

        nl_pred_ret = model.predict(test_x)
        nl_pred_ret = pd.Series(nl_pred_ret, index=test_x.index)
        nl_pred_ret = nl_pred_ret/nl_pred_ret.std()

        nl_corr = nl_pred_ret.corr(test_y)

        lm_model = LinearRegression(fit_intercept=False)
        lm_model.fit(train_x, train_y)
        #beta = (train_x.iloc[:,0]).corr(train_y.iloc[:,0]) * (train_y.iloc[:,0]).std() / (train_x[:,0]).std()
        l_pred_ret = lm_model.predict(test_x)
        l_pred_ret = pd.Series(l_pred_ret, index=test_x.index)
        l_pred_ret = l_pred_ret/l_pred_ret.std()

        l_corr = l_pred_ret.corr(test_y)

        comb_pred_ret = ( nl_pred_ret + l_pred_ret )/2.0
        comb_corr = comb_pred_ret.corr(test_y)


        nl_l_corr = nl_pred_ret.corr(l_pred_ret)


        l_ic_vec.append(l_corr)
        nl_ic_vec.append(nl_corr)

        comb_ic_vec.append(comb_corr)

        nl_l_corr_vec.append(nl_l_corr)

        model_vec.append(model)
        train_x_vec.append(train_x)

        #print( r_d,l_corr, nl_corr )

    df = pd.DataFrame()
    df['l_corr'] = l_ic_vec
    df['nl_corr'] = nl_ic_vec
    df['comb_ic'] = comb_ic_vec
    df['nl_l_corr'] = nl_l_corr_vec

    #df.index = rebalance_dates
    corr_s = df.mean(axis=0)
    corr_s.name = sig_name
    corr_df = corr_df.append(corr_s)
    print(sig_name)
    print(corr_s)

absacc
l_corr      -0.001650
nl_corr      0.007957
comb_ic      0.000713
nl_l_corr    0.212880
Name: absacc, dtype: float64
beta
l_corr      -0.000832
nl_corr      0.002080
comb_ic      0.001803
nl_l_corr    0.347879
Name: beta, dtype: float64
cashdebt
l_corr      -0.008054
nl_corr      0.023534
comb_ic      0.012288
nl_l_corr   -0.214198
Name: cashdebt, dtype: float64
chcsho
l_corr       0.000928
nl_corr      0.007460
comb_ic      0.005753
nl_l_corr    0.294077
Name: chcsho, dtype: float64


KeyboardInterrupt: 

In [ ]:
corr_df

In [ ]:
#ind = 189
#model = model_vec[ind]
#X = np.arange(0.0,1.01,0.01)
#X = X.reshape((X.shape[0],1))
#pred_y = model.predict(X)
#pylab.plot(X[:,0], pred_y)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
rcParams['font.size']=  15
corr_df[['l_corr', 'nl_corr']].iloc[::10].plot(kind='bar')